In [21]:
# IBM Data Science
# https://github.com/learningsam20/IBMDataScience
# https://www.coursera.org/learn/applied-data-science-capstone/peer/I1bDq/segmenting-and-clustering-neighborhoods-in-toronto
#
from bs4 import BeautifulSoup
import geocoder
import pandas as pd
import urllib3
import requests

In [50]:
! pip install beautifulsoup4
! pip install geocoder
! pip install pandas
! pip install urllib3
! pip install requests

In [44]:
# set defaults for pandas dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# parse the wiki
wikiurl = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
http = urllib3.PoolManager()
html = http.request('GET', wikiurl).data.decode('utf-8')

C:\Users\Sameer\AppData\Roaming\Python\Python37\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [46]:
# parse the DOM using beautiful soup
soup = BeautifulSoup(html)
# find table and get all rows within the table
datatable=soup.find("table",attrs={"class":"wikitable"}).findAll("tr")
# create the dataframe for storing parsed data
df=pd.DataFrame(columns=("PostalCode","Borough","Neighborhood"))
alldata=[]
for r in datatable:
    d=r.findAll("td")
    v={}
    # only include rows having Borough not equal to not assigned
    if len(d)>0 and d[1].text.strip() !="Not assigned":
        v["PostalCode"]=d[0].text.strip()
        v["Borough"]=d[1].text.strip()
        # copy Borough to Neighborhood if it is not assigned ot empty
        v["Neighborhood"]=d[1].text.strip() if d[2].text.strip()=="Not assigned" else d[2].text.strip()
        alldata.append(v)
df=pd.DataFrame(alldata)
print("dataframe loaded with " + str(len(df)) + " records")

dataframe loaded with 103 records


In [47]:
# group by the PostalCode and Borough and combined Neighborhood
dfg=pd.DataFrame(df.groupby(["PostalCode","Borough"],as_index=False).apply(lambda x: "%s" % ', '.join(x.Neighborhood)).reset_index())
dfg.columns=["PostalCode","Borough","Neighborhood"]
print(dfg.head())

  PostalCode      Borough                            Neighborhood
0        M1B  Scarborough                          Malvern, Rouge
1        M1C  Scarborough  Rouge Hill, Port Union, Highland Creek
2        M1E  Scarborough       Guildwood, Morningside, West Hill
3        M1G  Scarborough                                  Woburn
4        M1H  Scarborough                               Cedarbrae


In [48]:
#df.reset_index(True)
print(dfg.shape)
#print(dfg.loc[dfg['PostalCode'] == "M5A"])
#print(dfg.loc[dfg['PostalCode'] == "M9V"])

(103, 3)
